# Team Performance Analytics

This notebook analyzes team member performance metrics using data from Google Sheets. We'll create various visualizations to understand:
- Daily performance patterns
- Weekly trends and patterns
- Monthly performance overview
- Key Performance Indicators (KPIs)

First, let's set up our environment and import the required libraries.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import calendar

# Set style for better visualizations
plt.style.use('seaborn')
sns.set_palette("husl")

# Configure pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Function to connect to Google Sheets and fetch data
def fetch_sheets_data(spreadsheet_id):
    """
    Fetch data from Google Sheets using the credentials and spreadsheet ID
    Returns a dictionary of dataframes, one for each team member
    """
    import os
    from google.oauth2.credentials import Credentials
    from googleapiclient.discovery import build
    
    # Initialize credentials and service
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    creds = None
    
    # Get credentials from the JSON file
    if os.path.exists('../credentials.json'):
        from google.oauth2 import service_account
        creds = service_account.Credentials.from_service_account_file(
            '../credentials.json', scopes=SCOPES)
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    
    # Get all sheet names
    sheet_metadata = sheet.get(spreadsheetId=spreadsheet_id).execute()
    sheets = sheet_metadata.get('sheets', '')
    
    data_dict = {}
    
    # Fetch data from each sheet
    for sheet_item in sheets:
        sheet_name = sheet_item['properties']['title']
        range_name = f'{sheet_name}!A1:I'
        
        result = sheet.values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        if values:
            # Convert to dataframe
            df = pd.DataFrame(values[1:], columns=values[0])
            # Convert date strings to datetime
            df['Date'] = pd.to_datetime(df['Date'])
            # Convert numeric columns
            numeric_columns = ['Tickets Assigned', 'Tickets Resolved', 
                             'SLA Breaches', 'Resopond Tickets', 
                             'Client Interactions']
            for col in numeric_columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            
            data_dict[sheet_name] = df
            
    return data_dict

# Fetch data from Google Sheets
SPREADSHEET_ID = "1vl5gTB6OkLVSvYvnCfLwHW_FyjKUinkiKxav-5zaA80"
team_data = fetch_sheets_data(SPREADSHEET_ID)

# Combine all data into a single dataframe
df_combined = pd.concat([df.assign(Member=member) for member, df in team_data.items()], 
                       ignore_index=True)

print("Data loaded successfully!")
df_combined.head()

# Daily Performance Analysis

Let's create visualizations to analyze daily performance metrics for each team member.

In [ ]:
# Daily Tickets Analysis - Bar Chart
plt.figure(figsize=(15, 6))

# Group by date and member, calculate mean for assigned and resolved tickets
daily_tickets = df_combined.groupby(['Date', 'Member']).agg({
    'Tickets Assigned': 'sum',
    'Tickets Resolved': 'sum'
}).reset_index()

# Create bar chart using plotly
fig = px.bar(daily_tickets, x='Date', y=['Tickets Assigned', 'Tickets Resolved'],
             color='Member', barmode='group',
             title='Daily Tickets Assigned vs Resolved by Team Member',
             labels={'value': 'Number of Tickets', 'variable': 'Metric'})

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Number of Tickets",
    legend_title="Team Member & Metric",
    template='plotly_white'
)

fig.show()

# Daily SLA Breaches Trend
plt.figure(figsize=(15, 6))

daily_sla = df_combined.groupby(['Date', 'Member'])['SLA Breaches'].sum().reset_index()

fig = px.line(daily_sla, x='Date', y='SLA Breaches', color='Member',
              title='Daily SLA Breaches Trend by Team Member',
              labels={'SLA Breaches': 'Number of SLA Breaches'})

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Number of SLA Breaches",
    template='plotly_white'
)

fig.show()

# Stacked Bar: Responded Tickets vs Client Interactions
plt.figure(figsize=(15, 6))

daily_interactions = df_combined.groupby(['Date', 'Member']).agg({
    'Resopond Tickets': 'sum',
    'Client Interactions': 'sum'
}).reset_index()

fig = px.bar(daily_interactions, x='Date', 
             y=['Resopond Tickets', 'Client Interactions'],
             color='Member', 
             title='Daily Responded Tickets vs Client Interactions',
             barmode='stack')

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Count",
    legend_title="Team Member",
    template='plotly_white'
)

fig.show()

# Weekly Performance Analysis

Now let's analyze the weekly patterns in team performance.

# Team Performance Analytics Dashboard

This notebook analyzes team member performance metrics using data from Google Sheets. We'll create various visualizations to understand daily, weekly, and monthly patterns in:
- Ticket assignments and resolutions
- SLA breaches
- Client interactions
- Response times

## Setup and Dependencies

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from datetime import datetime, timedelta
import calendar

# Set style for matplotlib
plt.style.use('seaborn')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
# Load and preprocess data
def load_and_preprocess_data(spreadsheet_id):
    # Load data from Google Sheets
    # For now, we'll use sample data
    data = pd.DataFrame({
        'Date': pd.date_range(start='2025-01-01', periods=100),
        'Name': np.random.choice(['Winnish Allwin G J', 'Vuppu Reddy Kalyan', 'Sam J', 'Kiran Kumar'], 100),
        'Email': ['email@example.com'] * 100,
        'Tickets_Assigned': np.random.randint(5, 20, 100),
        'Tickets_Resolved': np.random.randint(3, 15, 100),
        'SLA_Breaches': np.random.randint(0, 5, 100),
        'Respond_Tickets': np.random.randint(2, 10, 100),
        'Client_Interaction': np.random.randint(1, 8, 100),
        'Remarks': np.random.choice(['Bug Fix', 'Feature Request', 'Support', 'Documentation'], 100)
    })
    
    # Convert date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Add derived columns
    data['Week'] = data['Date'].dt.isocalendar().week
    data['Month'] = data['Date'].dt.month
    data['Day_of_Week'] = data['Date'].dt.day_name()
    data['Resolution_Rate'] = (data['Tickets_Resolved'] / data['Tickets_Assigned'] * 100).round(2)
    
    return data

# Load the data
df = load_and_preprocess_data("your_spreadsheet_id")